In [2]:
#Comment out below two lines to install h5py
#import sys
#!{sys.executable} -m pip install h5py

import tables
import os
import h5py

orignial_das_file="./das_example_w_earthquake.h5"
compressed_das_file="./das_example_w_earthquake_compressed.h5"
das_f = h5py.File(orignial_das_file, 'r')
das_dset = das_f['Acoustic']

print(das_dset.shape)
print(das_dset.dtype)

das_f_compressed = h5py.File(compressed_das_file, "w")
das_dset_compressed = das_f_compressed.create_dataset("Acoustic", chunks=das_dset.shape, data=das_dset, compression=62016, compression_opts=(0,1,60000,6912))
das_dset_compressed=das_dset
das_f_compressed.close()
das_f.close()

##inspect the result
!h5dump -pH ./das_example_w_earthquake_compressed.h5


(60000, 6912)
int16
HDF5 "./das_example_w_earthquake_compressed.h5" {
GROUP "/" {
   DATASET "Acoustic" {
      DATATYPE  H5T_STD_I16LE
      DATASPACE  SIMPLE { ( 60000, 6912 ) / ( 60000, 6912 ) }
      STORAGE_LAYOUT {
         CHUNKED ( 60000, 6912 )
         SIZE 602942007 (1.376:1 COMPRESSION)
      }
      FILTERS {
         USER_DEFINED_FILTER {
            FILTER_ID 62016
            COMMENT TurboPFor-Integer-Compression: https://github.com/dbinlbl/H5TurboPFor
            PARAMS { 0 1 60000 6912 }
         }
      }
      FILLVALUE {
         FILL_TIME H5D_FILL_TIME_ALLOC
         VALUE  H5D_FILL_VALUE_DEFAULT
      }
      ALLOCATION_TIME {
         H5D_ALLOC_TIME_INCR
      }
   }
}
}


In [3]:
import os
!git clone https://github.com/powturbo/TurboPFor-Integer-Compression.git
!cd TurboPFor-Integer-Compression ; make
!cmake . ; make clean; make install
!source setup.sh




fatal: destination path 'TurboPFor-Integer-Compression' already exists and is not an empty directory.
make: Nothing to be done for `all'.
-- Configuring done
CMake Warning (dev):
  Policy CMP0042 is not set: MACOSX_RPATH is enabled by default.  Run "cmake
  --help-policy CMP0042" for policy details.  Use the cmake_policy command to
  set the policy and suppress this warning.

  MACOSX_RPATH is not specified for the following targets:

   turbopfor_h5_plugin_shared

This warning is for project developers.  Use -Wno-dev to suppress it.

-- Generating done
-- Build files have been written to: /Users/dbin/work/H5TurboPFor
[ 50%] Building C object CMakeFiles/turbopfor_h5_plugin_shared.dir/turbopfor_h5plugin.c.o
[100%] Linking C shared library libH5Zturbopfor.dylib
[100%] Built target turbopfor_h5_plugin_shared
Install the project...
-- Install configuration: ""
-- Installing: /Users/dbin/work/H5TurboPFor/././build/libH5Zturbopfor.dylib
